<a href="https://colab.research.google.com/github/ansar-mazhar/ML-Task-1/blob/main/ML_Task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Content Based Recommendation System

In [1]:
#import libraries & load dataset
import pandas as pd
path = "/content/oldies_60s_top_artists_tracks.csv"
df = pd.read_csv(path)
df.head()


,Artist,Track Name,Popularity,Duration (ms),Track ID
0,The Beatles,Here Comes The Sun - Remastered 2009,87,185733,6dGnYIeXmHdcikdzNNDMm2
1,The Beatles,Yesterday - Remastered 2009,80,125666,3BQHpFgAp4l80e1XslIjNI
2,The Beatles,Come Together - Remastered 2009,80,259946,2EqlS6tkEnglzr7tkKAAYD
3,The Beatles,Twist And Shout - Remastered 2009,80,155226,5ZBeML7Lf3FMEVviTyvi8l
4,The Beatles,Let It Be - Remastered 2009,80,243026,7iN1s7xHE4ifF5povM6A48


# Data Preprocessing

In [2]:
df['Duration (sec)'] = df['Duration (ms)']/1000
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df['Popularity Scaled'] = scaler.fit_transform(df[['Popularity']])
df = pd.get_dummies(df, columns=["Artist"], drop_first=True)
df.head()

,Track Name,Popularity,Duration (ms),Track ID,Duration (sec),Popularity Scaled,Artist_Bob Dylan,Artist_Booker T. & The MG’s,Artist_Chubby Checker,Artist_Cream,...,Artist_The Marvelettes,Artist_The Monkees,Artist_The Rolling Stones,Artist_The Shirelles,Artist_The Supremes,Artist_The Temptations,Artist_The Ventures,Artist_The Who,Artist_The Yardbirds,Artist_Wilson Pickett
0,Here Comes The Sun - Remastered 2009,87,185733,6dGnYIeXmHdcikdzNNDMm2,185.733,0.966667,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,Yesterday - Remastered 2009,80,125666,3BQHpFgAp4l80e1XslIjNI,125.666,0.888889,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Come Together - Remastered 2009,80,259946,2EqlS6tkEnglzr7tkKAAYD,259.946,0.888889,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Twist And Shout - Remastered 2009,80,155226,5ZBeML7Lf3FMEVviTyvi8l,155.226,0.888889,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Let It Be - Remastered 2009,80,243026,7iN1s7xHE4ifF5povM6A48,243.026,0.888889,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# Training Recommendation Model

In [3]:
from sklearn.metrics.pairwise import cosine_similarity

# Drop any non-numeric columns (like 'Track Name' and original 'Popularity')
non_numeric = ['Track Name', 'Popularity', 'Track ID']
features = df.drop(columns=[col for col in non_numeric if col in df.columns])

# Compute cosine similarity
similarity_matrix = cosine_similarity(features)

print("Similarity matrix shape:", similarity_matrix.shape)



Similarity matrix shape: (484, 484)


# Recommendation Function

In [5]:
def recommend_tracks(track_name, df, similarity_matrix, n_recommendations=5):
    if track_name not in df['Track Name'].values:
        print("Track not found in the dataset.")
        return []

    # Find the index of the given track
    idx = df[df['Track Name'] == track_name].index[0]

    # Get similarity scores for this track
    similarity_scores = list(enumerate(similarity_matrix[idx]))

    # Sort by similarity (excluding the track itself)
    sorted_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[1:n_recommendations+1]

    # Get recommended track names
    recommended_indices = [i[0] for i in sorted_scores]
    recommended_tracks = df.iloc[recommended_indices]["Track Name"].values

    print(f"Recommendations for '{track_name}':")
    for i, rec in enumerate(recommended_tracks, 1):
        print(f"{i}. {rec}")

    return recommended_tracks


recommend_tracks("Here Comes The Sun - Remastered 2009", df, similarity_matrix)



Recommendations for 'Here Comes The Sun - Remastered 2009':
1. Something - Remastered 2009
2. Twist And Shout - Remastered 2009
3. In My Life - Remastered 2009
4. Let It Be - Remastered 2009
5. Blackbird - Remastered 2009


array(['Something - Remastered 2009', 'Twist And Shout - Remastered 2009',
       'In My Life - Remastered 2009', 'Let It Be - Remastered 2009',
       'Blackbird - Remastered 2009'], dtype=object)